In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import os
import time

from officelib.xllib import *

In [2]:
dl = os.path.expanduser("~\\downloads")
matrix_report_file = dl + "\\matrix-report.xlsx"
report_file = matrix_report_file.replace("report.xlsx", "dl-in.xlsx")

def del_old_reports():
    for file in (matrix_report_file, report_file):
        try:
            os.remove(file)
        except FileNotFoundError:
            pass  # fine with me

In [3]:
class wait_for_handle_if:
    def __init__(self, first_condition, handle_condition, func):
        self.first = first_condition
        self.handle = handle_condition
        self.func = func
        
    def __call__(self, driver):
        try:
            elem = self.handle(driver)
        except WebDriverException:
            pass
        else:
            self.func(elem)
        
        return self.first(driver)
        

In [4]:
def dl_mtx_report():
    with webdriver.Chrome() as driver:
        _dl_mtx_report(driver)
        
def _dl_mtx_report(driver):
    driver.get("https://pbsbiotech.helixalm.cloud/ttweb/#default/10/previewReport/6")

    # username = driver.find_element_by_css_selector("#login-username [name='login-username']")
    wait = WebDriverWait(driver, 15, 0.5)

    # elements = [
    #     (By.CSS_SELECTOR, "#login-username [name='login-username']"),
    #     (By.CSS_SELECTOR, "#login-password [name='login-password']"),
    # #     (By.ID, "sc6643-label")
    #     (By.CSS_SELECTOR, "label [title='Log in to Helix ALM']")
    # ]

    username = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#login-username [name='login-username']")))
    password = driver.find_element_by_css_selector("#login-password [name='login-password']")
    log_in = driver.find_element_by_css_selector("label[title='Log in to Helix ALM']")

    # sometimes this doesn't finish loading so wait juuuust a tad
    # note: also switched the wait condition so not sure if still necessary
    time.sleep(0.1)
    username.send_keys("nstarkweather@pbsbiotech.com")
    password.send_keys("sadbearbonebar")
    
    current_handles = driver.window_handles
    
    log_in.click()

    # clear the alert dialog if it appears for existing login
    # note: no clean way to get dialog button reference
    
    cond = wait_for_handle_if(EC.new_window_is_opened(current_handles), 
                              EC.visibility_of_element_located((By.ID,"sc6854")),
                             lambda elem: elem.click())
    
    wait.until(cond)
    login_handle, report_handle = driver.window_handles
    driver.switch_to.window(login_handle)
    driver.close()
    driver.switch_to.window(report_handle)
    driver.execute_script("javascript:ExportMatrixToExcel('maintable')")

    timeout = 10  # seconds
    end = time.time() + timeout
    report_file = matrix_report_file.replace("report.xlsx", "dl-in.xlsx")

    # no wait to wait for the file to download intentionally
    # easiest workaround is to try and rename it over and over
    # until it lets us (file exists & is no longer open)

    while True:
        try:
            os.rename(matrix_report_file, report_file)
        except (FileNotFoundError, PermissionError):
            if time.time() > end:
                raise
            time.sleep(0.1)

        else:
            break

In [5]:
def _stripify(c):
    if c is None: return ""
    t = c.strip().split(" ")
    return " ".join(t[2:])

def _unpack(row):
    return [c or "" for c in row]

def _parse_row(row):
    feature, fd, req, test = _unpack(row)
    feature = _stripify(feature)
    test = _stripify(test)
    return feature, test

def _superfit(c):
    c.ColumnWidth = 255
    c.AutoFit()

def _worksheet2(wb):
    if wb.Worksheets.Count < 2:
        return wb.Worksheets.Add()
    return wb.Worksheets(2)
    
def xfer(xl, wb, ws, cells, cr):
    # get data for copy to sheet2
    tl = cr("A2")
    br = tl.End(xlc.xlDown).GetOffset(0, 3)

    # (Feature, Functional Design, System Requirements, Test Cases)
    data = cr(tl, br).Value2

    from collections import defaultdict

    # paste data
    ws2 = _worksheet2(wb)
    cr2 = ws2.Cells.Range
    ws2.UsedRange.Clear()

    fmap = defaultdict(set)
    for row in data:
        feature, test = _parse_row(row)
        fmap[feature].add(test)
    
    paste_data = []
    for feature, tests in fmap.items():
        tests_list = "\n".join(sorted(filter(None, tests)))
        if not tests_list:
            pass
#             continue
        row = (feature, tests_list)
        paste_data.append(row)
    
    tl2 = cr2("A2")
    br2 = tl2.GetOffset(len(paste_data) - 1, len(paste_data[0]) - 1)
    
    cr2(tl2, br2).Value2 = paste_data
    
    cr2("A1:B1").Value2 = [("Feature", "Factory & OQ Test Sections")]
    
    for col in (1,2):
        _superfit(ws2.Columns(col))
    
    ur = ws2.UsedRange
    ur.Rows.AutoFit()
    ur.VerticalAlignment = xlc.xlTop
    

In [6]:
def import_helix_mtrx(xl, filename):
    wb = xl.Workbooks.Open(filename)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

    with screen_lock(xl):
        xfer(xl, wb, ws, cells, cr)
    


In [10]:
def close_wb_if_user_is_dumbass(xl, filename):
    for wb in xl.Workbooks:
        if wb.FullName == filename:
            print(">:( >:( >:(")
            wb.Close(False)
            return

def main():
    xl = Excel()
    close_wb_if_user_is_dumbass(xl, report_file)
    del_old_reports()
    dl_mtx_report()
    with screen_lock(xl):
        import_helix_mtrx(xl, report_file)

In [11]:
wb = xl.Workbooks(report_file)

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147352565), None)